# To do 
- [x] Add translation of the input phrase and of the words table
- [x] Add text to speech method
- [ ] Add words learned attribute

# Environment

In [63]:
# nlp library
import spacy
# de, en nlp models
import de_core_news_sm, en_core_web_sm
# data wrangling
import pandas as pd
# translation
from google.cloud import translate_v2
# create google application credentials env variable
import os
# text to speech
from gtts import gTTS
from IPython.display import Audio

# Class

In [64]:
class LearningGerman:
    def __init__(self, text='testen'):
        self.text = text
        self.words, self.lemma, self.pos, self.details = ([] for c in range(4))
        self.table_ = pd.DataFrame()
        self.client = translate_v2.Client()
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '.projdata/credentials.json'
        self.language = self.client.detect_language(self.text)['language']
        if self.language == 'de':
            self.to = 'en'
        elif self.language == 'en':
            self.to = 'de'
        self.model = 'nmt'
    
    @property
    def table(self):
        if self.table_.empty:
            print('To create a complete vocab chart, please call nlp()!')
        else:
            return self.table_
    
    @staticmethod
    def drop_punct(table):
        if table['pos'].str.contains('PUNCT').any():
            table.drop(table[table.pos == 'PUNCT'].index, inplace=True)
    
    def translate(self, words=False):
        if words == False:
            return self.client.translate(self.text, source_language=self.language, target_language=self.to, 
                             model=self.model, format_='text')['translatedText']
        else:
            return [self.client.translate(word, source_language=self.language, target_language=self.to, 
                             model=self.model, format_='text')['translatedText'] for word in self.words]
    
    def nlp(self, visualize=False):
        if self.language == 'de':
            nlp = de_core_news_sm.load()
            column_names = ['deutsche']
        elif self.language == 'en':
            nlp = en_core_web_sm.load()
            column_names = ['english']
        doc = nlp(self.text)
        columns = ['lemma', 'pos', 'details']
        column_names.extend(columns)
        for token in doc:
            self.words.append(token.text)
            self.lemma.append(token.lemma_.lower())
            self.pos.append(token.pos_)
            self.details.append(spacy.explain(token.tag_))
        words_table = pd.DataFrame(data=zip(self.words, self.lemma, self.pos, self.details), columns=column_names)
        words_table['translation'] = self.translate(words=True)
        self.drop_punct(words_table)
        if visualize == True:
            spacy.displacy.render(doc, style='dep')
        print(f'\033[1;33m{self.text}\033[m -> {self.translate()}')
        self.table_ = words_table
        return words_table
    
    def say(self, translation=False, slowmode=False, autoplay=True):
        if translation == False:
            print(f'\033[1;33m{self.language}')
            speech = gTTS(self.text, lang=self.language, slow=slowmode)
            speech.save('.projdata/speech.mp3')
        else:
            print(f'\033[1m{self.to}')
            speech = gTTS(self.translate(), lang=self.to, slow=slowmode)
            speech.save('.projdata/speech.mp3')
        return Audio('.projdata/speech.mp3', autoplay=autoplay)